In [38]:
import torch
from monai.networks.nets import SegResNet, UNet
import torch
import torch.nn as nn

# SegResNet
model = SegResNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=105,  
    init_filters=32,   
    dropout_prob=0.2,
    blocks_down=(1, 2, 2, 4),
    blocks_up=(1, 1, 1),
)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
state_dict = torch.load('/Users/keyi/Desktop/pre_trainedwholeBody_ct_segmentation/models/model.pt',map_location=torch.device('cpu'), weights_only=True)
model.load_state_dict(state_dict, strict=True)
model.out_channels = 63

for name, param in model.named_parameters():
    if "up_layers" in name or "up_samples" in name or "conv_final" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
model.conv_final[2].conv = nn.Conv3d(32, 63, kernel_size=(1, 1, 1), stride=(1, 1, 1))

Anatomy_net = model.to(device)
test_input = torch.randn(1, 1, 128, 128, 128).to(device)

# Set model to evaluation mode
model.eval()

# Run inference
with torch.no_grad():  # Disable gradient calculation
    output = model(test_input)
    print(f"input shape: {test_input.shape}")
    print(f"output shape: {output.shape}")
# cross_stitch_matrix = nn.Parameter(torch.Tensor([[0.9,0.1],[0.4,0.6]]))
# cross_stitch_matrix
# pathology_net = UNet(
#             spatial_dims=3,
#             in_channels=2, # CT+ SUV
#             out_channels=2,
#             channels=[16, 32, 64, 128],
#             strides=[2, 2, 2, 2],
#             num_res_units=2,
#             norm='batch'
#         )

# print(Anatomy_net)
# print(pathology_net)
# for i, layer in enumerate(Anatomy_net.model):
#     print(f"Layer {i}: {layer}")
print(Anatomy_net)

input shape: torch.Size([1, 1, 128, 128, 128])
output shape: torch.Size([1, 63, 128, 128, 128])
SegResNet(
  (act_mod): ReLU(inplace=True)
  (convInit): Convolution(
    (conv): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
  )
  (down_layers): ModuleList(
    (0): Sequential(
      (0): Identity()
      (1): ResBlock(
        (norm1): GroupNorm(8, 32, eps=1e-05, affine=True)
        (norm2): GroupNorm(8, 32, eps=1e-05, affine=True)
        (act): ReLU(inplace=True)
        (conv1): Convolution(
          (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
        (conv2): Convolution(
          (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
      )
    )
    (1): Sequential(
      (0): Convolution(
        (conv): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      )
      (1): ResBlock(
        